In [ ]:
import os, dynamixel, time, random
import sys
from serial import Serial

# The number of Dynamixels on our bus.
nServos = 11

# Set your serial port accordingly.
if os.name == "posix":
    portName = "/dev/ttyUSB0"
else:
    portName = "COM5"
    
# Default baud rate of the USB2Dynamixel device.
baudRate = 1000000

serial = dynamixel.SerialStream( port=portName, baudrate=baudRate, timeout=1)
net = dynamixel.DynamixelNetwork( serial )

print("Scanning Dynamixel network..."),
sys.stdout.flush()

net.scan( 1, nServos )
print(" Done.")

myActuators = list()

print "Detected Dynamixel ids : ",
for dyn in net.get_dynamixels():
    print dyn.id,
    myActuators.append(dyn)
print

In [ ]:
for actuator in myActuators:
    actuator.moving_speed = 255
    actuator.synchronized = True
    actuator.torque_enable = True
    actuator.torque_limit = 800
    actuator.max_torque = 800

In [ ]:
myActuators[0].goal_position=102
"myActuators[1].goal_position=0"
net.synchronize()
time.sleep(0.5)


In [ ]:
flag = 0
decompte = 0
erreur = 0
while myActuators[0].goal_position<614 and flag == 0:
    decompte = decompte + 1
    for actuator in myActuators:
        actuator.goal_position = actuator.goal_position+17
    net.synchronize()
    for actuator in myActuators:
        actuator.read_all()
        time.sleep(0.01)
    for actuator in myActuators:
        #print actuator.cache[dynamixel.defs.REGISTER['Id']], actuator.cache[dynamixel.defs.REGISTER['CurrentPosition']]
        "print actuator.id, actuator.current_position, actuator.goal_position"
        if actuator.current_position == actuator.goal_position:
            flag = 1
    time.sleep(0.5)
    if actuator.current_position != actuator.goal_position:
        print "erreur", actuator.current_position, actuator.goal_position
        erreur = erreur + 1
print decompte, erreur

In [ ]:
serial.close()


In [21]:
import os, time,dynamixel, random
import sys
from serial import Serial

class Moteurs(object):
    
    def __init__(self):
        # The number of Dynamixels on our bus.
        nServos = 10
        # Set your serial port accordingly.
        if os.name == "posix":
            portName = "/dev/tty.usbserial-A94N75T1"
        else:
            portName = "COM5"
        # Default baud rate of the USB2Dynamixel device.
        baudRate = 1000000
        
        self.serial = dynamixel.SerialStream( port=portName, baudrate=baudRate, timeout=1)
        self.net = dynamixel.DynamixelNetwork( self.serial )
        
        sys.stdout.flush()
        
        self.net.scan( 0, nServos )
        myActuators = list()
        
        for dyn in self.net.get_dynamixels():
            myActuators.append(dyn)
        self.Moteurs=myActuators
    
    def conversion_angle_interval(self, theta ):
        """ Take an angle and convert it into a cervomotor interval """
        minimum_angle = 300 / 1024. #equivalent in ° of a motor interval
        targeted_interval = round(theta / minimum_angle) #interger equivalent of the targeted angle in motor interval
        return targeted_interval
    
    def conversion_interval_angle(self, interval ):
        """ Take an angle and convert it into a cervomotor interval """
        minimum_angle = 300 / 1024. #equivalent in ° of a motor interval
        angle = interval * minimum_angle #interger equivalent of the targeted angle in motor interval
        return angle
        
        
    def move_motor(self, ID, theta_f, V = 255):
        """ Take a motor ID, an ordered angle and the motor speed and make it move
        Return the residual error on the move"""
        self.Moteurs[ID].goal_position = int(self.conversion_angle_interval( theta_f ))
        self.net.synchronize()
        time.sleep(0.2)
        self.Moteurs[ID].read_all() #read all the properties of the cervomotor
        while self.Moteurs[ID].current_speed!=0:
            time.sleep(0.1)#Need time to move
            print(self.Moteurs[ID].current_speed)
        self.Moteurs[ID].read_all() #read all the properties of the cervomotor
        residual_error = self.Moteurs[ID].current_position - self.conversion_angle_interval (theta_f)
        return self.conversion_interval_angle (residual_error)
    
    def close(self):
        self.serial.close()

In [23]:
moteurs=Moteurs()


In [26]:
moteurs.move_motor(0,10)

-312
-304
-300
-180
20
0


-0.5859375

In [22]:
moteurs.close()